Homework 5: machine teaching for kNN
---
Ruixuan Tu (ruixuan@cs.wisc.edu)

29 March 2023

# Preparation

In [ ]:
data_file = "hw5bdata.txt"

## Setup

In [ ]:
using SHA
using Random
using Tidier
using Gadfly
using CSV
using DataFrames
using Combinatorics
using StatsBase
using JSONTables

cd("/Users/turx/Projects/machine-teaching-23sp/hw05-machine-teaching-knn")
Random.seed!(sum(sha256("machine-teaching")))

## Read and Plot Data

In [ ]:
pool = CSV.read(data_file, delim=' ', header=[:x1, :x2, :y], DataFrame)

In [ ]:
plot(pool, x=:x1, y=:x2, color=:y, Geom.point, Scale.color_discrete)

## Constants for Problem Setting in Homework Section 4

In [ ]:
k = 1  # number of nearest neighbors
Z = @chain pool begin  # superset of all teaching sets
  @rename(x1p = x1, x2p = x2, yp = y)
end
enum_upper = 3  # n_star for enum, threshold for the number of teaching examples
greedy_upper = 20  # n_star for greedy, threshold for the number of teaching examples

## Shared functions

### kNN

In [ ]:
# Implement the kNN classifier for arbitrary k and arbitrary number of dimensions
# train_x: training data matrix, each row is a data point
# train_y: training label vector
# test_x: test data matrix, each row is a data point
# k: number of nearest neighbors
# return: predicted labels y-hat for test_x
function knn_classifier(train_x, train_y, test_x, k)
  test_y = zeros(Float64, size(test_x, 1))
  for i in 1:size(test_x, 1)
    dist = [sqrt(sum((train_x[j, ] - test_x[i, ]) ^ 2)) for j in 1:size(train_x, 1)]
    knn = sortperm(dist)[1:k]
    test_y[i] = mode(train_y[knn])
  end
  return test_y
end

# Predict the label of a single point using kNN classifier
# x1: vector of first dimension of data points
# x2: vector of second dimension of data points
# y: vector of labels
# x1p: position of the first dimension of the point to be labeled
# x2p: position of the second dimension of the point to be labeled
# k: number of nearest neighbors
# return: predicted label y-hat for the point (x1p, x2p)
function knn_classifier_single(x1, x2, y, x1p, x2p, k)
  train_x = vcat(x1', x2')'
  train_y = y
  test_x = [x1p x2p]
  return knn_classifier(train_x, train_y, test_x, k)[1]
end

### Approximation and Sampling

In [ ]:
# Binary loss function
# y: vector of true labels
# yp: vector of predicted labels
# return: binary loss
function loss_bin(y, yp)
  return sum(y .!= yp) / length(y)
end

# Metric between functions f and g defined by equation (3)
# Z: data frame (x1p, x2p, yp), x1p and x2p are positions of the points to be labeled, yp is the expected label
# f: function f, the predictor with arguments x1p, x2p, returning yp (i.e., y-hat)
# l: loss function, taking two vectors of labels and returning a scalar of total loss
function dist(Z, f, l)
  m = size(Z, 1)
  pred = f.(Z.x1p, Z.x2p)
  d = l(Z.yp, pred)
  return d
end

### OS

In [ ]:
# Time the execution of an expression
# return: tuple of (time in seconds, return value of f)
macro timed_run(expr)
    return quote
        local t0 = time()
        local ret = $(esc(expr))
        local t1 = time()
        ((t1 - t0), ret)
    end
end

# Run the function and save or load the resulting DataFrame from a csv file
# expr: expression to be run
# file: file to save or load the result
# return: DataFrame
macro saved_run(expr, file)
  return quote
    if isfile($file)
      CSV.read($file, DataFrame)
    else
      local df = $expr
      CSV.write($file, df)
      df
    end
  end
end

# Enumeration

In [ ]:
function enum_timed(m)
  best_d = Inf
  best_subset = nothing
  index_subsets = with_replacement_combinations(1:size(Z, 1), m)
  counter = 0
  for index_subset in index_subsets
    subset = pool[index_subset, :]
    counter += 1
    g = (x1p, x2p) -> knn_classifier_single(subset.x1, subset.x2, subset.y, x1p, x2p, k)
    d = dist(Z, g, loss_bin)
    if d < best_d
      best_d = d
      best_subset = subset
    end
  end
  return (counter, best_d, best_subset)
end

# Implementation of the enumeration algorithm
# Reporting:
# 1. the number of teaching sets of that size that you have to search through;
# 2. number of seconds it takes for that size;
# 3. d(kNN(D), g) of the best teaching set of that size;
# 4. plot the best teaching set D-hat in relation to P (i.e. plot both, but use different symbols for D-hat)
function enum_run()
  results = DataFrame(
    subset_sz = Int64[],
    subset_n = Int64[],
    time = Float64[],
    d = Float64[],
    subset = String[],
  )
  for m in 1:enum_upper
    (t, r) = @timed_run enum_timed(m)
    rs = objecttable(r[3])
    push!(results, (m, r[1], t, r[2], rs))
  end
  return results
end

enum_results = @saved_run enum_run() "enum.csv"

Plots: negative integer $-x$ in color means the point $x-1$ is picked by the algorithm

In [ ]:
function enum_plot(m)
  best_subset = DataFrame(jsontable(enum_results.subset[m]))
  best_d = enum_results.d[m]
  best_subset.y = -best_subset.y .- 1
  return plot(
    layer(best_subset, x=:x1, y=:x2, color=:y, Geom.point),
    layer(pool, x=:x1, y=:x2, color=:y, Geom.point),
    Guide.title("Enumeration: Best Teaching Set vs Pool: m = $m, d = $best_d"),
    Scale.color_discrete
  )
end

for m in 1:enum_upper
  p = enum_plot(m)
  draw(SVG("enum_m$m.svg"), p)
  display(p)
end

# Greedy

In [ ]:
function greedy_timed(prev_D)
  best_d = Inf
  idx_l = []
  for i in 1:size(pool, 1)
    D = vcat(prev_D, i)
    subset = pool[D, :]
    g = (x1p, x2p) -> knn_classifier_single(subset.x1, subset.x2, subset.y, x1p, x2p, k)
    d = dist(Z, g, loss_bin)
    if d < best_d
      best_d = d
      idx_l = [i]
    elseif d == best_d
      push!(idx_l, i)
    end
  end
  idx_to_add = rand(idx_l)
  return (size(pool, 1), best_d, idx_to_add)
end

# Implementation of the greedy algorithm
# Reporting:
# 1. the number of teaching sets of that size that you have to search through;
# 2. number of seconds it takes for that size;
# 3. d(kNN(D), g) of the best teaching set of that size;
# 4. plot one figure for the last teaching set D-hat with size n* in relation to P
function greedy_run()
  results = DataFrame(
    subset_sz = Int64[],
    subset_n = Int64[],
    time = Float64[],
    d = Float64[],
    index = Int64[],
  )
  prev_D = []
  for m in 1:greedy_upper
    if m > 1
      push!(prev_D, results.index[m - 1])
    end
    (t, r) = @timed_run greedy_timed(m)
    push!(results, (m, r[1], t, r[2], r[3]))
  end
  return results
end

greedy_results = @saved_run greedy_run() "greedy.csv"

In [ ]:
selected_pool = pool[greedy_results.index, :]
selected_pool.y = -selected_pool.y .- 1
last_d = greedy_results.d[greedy_upper]

p = plot(
  layer(selected_pool, x=:x1, y=:x2, color=:y, Geom.point),
  layer(pool, x=:x1, y=:x2, color=:y, Geom.point),
  Guide.title("Greedy: Last Teaching Set vs Pool: m = $greedy_upper, d = $last_d"),
  Scale.color_discrete
)
draw(SVG("greedy_m$greedy_upper.svg"), p)
p